In [10]:
from word2number import w2n

# initializing string
test_str = "thirty thousand five hundred thirty-three point one"

# printing original string
print("The original string is : " + test_str)

# Convert numeric words to numbers
# Using word2number
res = w2n.word_to_num(test_str)

# printing result
print("The string after performing replace : " + str(res))


The original string is : thirty thousand five hundred thirty-three point one
The string after performing replace : 30533.1


In [1]:
import cv2
import mouse
import numpy as np
import math
import wx 

try:
    del app;
except:
    pass;
    
#function to reduce flickering
def isPointClose(x1,y1,x2,y2,scale):
    d=math.sqrt((x1-x2)**2+(y1-y2)**2);
    if d<=scale:     #length<=scale?
        return True;
    else :
        return False;
    
#if __name__ == "__main__":
cap=cv2.VideoCapture(0);#'http://192.168.0.101:4747/mjpegfeed');
bg=cv2.flip(cap.read()[1],1);
w=np.shape(bg)[1];
h=np.shape(bg)[0];
bg=bg[1:h-199,250:w].copy();
app=wx.App(False);
(sx,sy)=wx.GetDisplaySize();
########################

#font = cv2.FONT_HERSHEY_SIMPLEX;
#low_range=np.array([123,112,4]);      #hand color thresholds
#high_range=np.array( [250,180,124]);

########################

mouseOn=0;
while True:
    frame=cv2.flip(cap.read()[1],1);
    
    
    roi=frame[1:h-199,250:w].copy();
    temp_roi=roi.copy();
    
    fmask=cv2.absdiff(bg,roi,0);
    fmask=cv2.cvtColor(fmask,cv2.COLOR_BGR2GRAY);
    fmask=cv2.threshold(fmask,10,255,0)[1];
     ####### Morphological Processing #########
    fmask=cv2.erode(fmask,cv2.getStructuringElement(cv2.MORPH_ERODE,(2,2)),iterations=2);
    mask1=cv2.morphologyEx(fmask,cv2.MORPH_CLOSE,\
                           cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(4,4)));
    mask1=cv2.erode(mask1,cv2.getStructuringElement(cv2.MORPH_ERODE,(2,2)),iterations=2);
    cv2.imshow('mask1',mask1);
    fg_frame=cv2.bitwise_and(roi,roi,mask=mask1);
    cv2.imshow('fg_frame',fg_frame);
    
    gr_frame=cv2.cvtColor(fg_frame,cv2.COLOR_BGR2GRAY);
    gr_frame=cv2.blur(gr_frame,(10,10));
    bw_frame=cv2.threshold(gr_frame,50,255,0)[1];
    
    ############ Tracking the hand contour ################
    
    con=cv2.findContours(bw_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0];
    try:
        my_con=max(con,key=cv2.contourArea);
    except:
        my_con=np.array([[[1,0],[1,2],[2,3]]],dtype=np.int32);
        #pass;
    try:
        if cv2.contourArea(my_con)>90:
            
            hull=cv2.convexHull(my_con,True);
            
            leftmost = tuple(hull[hull[:,:,0].argmin()][0]) 
            rightmost = tuple(my_con[my_con[:,:,0].argmax()][0]) 
            topmost = tuple(hull[hull[:,:,1].argmin()][0]) 
            bottommost = tuple(my_con[my_con[:,:,1].argmax()][0])
            
            
            temp=bottommost[0]+30 #getting the bottom middle of the hand
            cv2.line(roi,topmost,(topmost[0],h-280),(0,242,225),2);
            cv2.line(roi,leftmost,(topmost[0],bottommost[1]-80),(0,242,225),2);
            
            cv2.circle(roi,topmost,5,(255,0,0),-1);
            cv2.circle(roi,leftmost,5,(0,120,255),-1);
            cv2.circle(roi,(temp,bottommost[1]),5,(230,0,255),-1);
            ###################### angle calculate #####################
            x1=topmost[0];y1=topmost[1];
            x2=bottommost[0]+20;y2=bottommost[1];
            x3=leftmost[0];y3=leftmost[1];
            m1=(y2-y1)/(x2-x1)
            m2=(y3-y2)/(x3-x2)
            tan8=math.fabs((m2-m1)/(1+m1*m2));
            angle=math.atan(tan8)*180/math.pi;
            ############################################################
            
            
            #angle = math.atan2(y2 - y1, x2 - x1) * 180.0 / math.pi;
            length=math.sqrt(math.pow((y2-y1),2)+math.pow((x2-x1),2))
            #print(angle);
            
            if length<50:
                continue
            
            ################### get original pixel location #############
            
            x=sx-((topmost[0]-50)*sx/(w-340));
            y=(topmost[1]*sy/(h-281));
            mouse.move(sx-x,y,absolute=True, duration=.1);
            
            
            
            cv2.putText(roi,str('%d,%d'%(sx-x,y)),topmost, cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            #################### Clicking the mouse ########################
            if angle<15:
                mouse.click(button='left');    # uncomment to activate the left click
                print('left clicked');
                pass
            else:
                pass
            
    except:
        pass;
    frame[1:h-199,250:w]=roi;
    cv2.rectangle(frame,(250,1),(w-1,h-200),(0,255,0),2);
    cv2.rectangle(frame,(300,1),(w-40,h-280),(255,0,0),2);
    cv2.imshow('frame',frame);
    if cv2.waitKey(2)==ord('r'):
        print('Background reset')
        bg=temp_roi;
    elif cv2.waitKey(2)==27:
        break;
#%%############# Releasing the resources ##############
cv2.destroyAllWindows();
cap.release();



In [5]:
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import pytesseract 
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
root = Tk()
root.title('TechVidvan Text from image project') 
newline= Label(root)
uploaded_img=Label(root)
scrollbar = Scrollbar(root)
scrollbar.pack( side = RIGHT, fill = Y )
def extract(path):
    Actual_image = cv2.imread(path)
    Sample_img = cv2.resize(Actual_image,(400,350))
    Image_ht,Image_wd,Image_thickness = Sample_img.shape
    Sample_img = cv2.cvtColor(Sample_img,cv2.COLOR_BGR2RGB)
    texts = pytesseract.image_to_data(Sample_img) 
    mytext=""
    prevy=0
    for cnt,text in enumerate(texts.splitlines()):
        if cnt==0:
            continue
        text = text.split()
        if len(text)==12:
            x,y,w,h = int(text[6]),int(text[7]),int(text[8]),int(text[9])
            if(len(mytext)==0):
                prey=y
            if(prevy-y>=10 or y-prevy>=10):
                print(mytext)
                Label(root,text=mytext,font=('Times',15,'bold')).pack()
                mytext=""
            mytext = mytext + text[11]+" "
            prevy=y
    Label(root,text=mytext,font=('Times',15,'bold')).pack()
def show_extract_button(path):
    extractBtn= Button(root,text="Extract text",command=lambda: extract(path),bg="#2f2f77",fg="gray",pady=15,padx=15,font=('Times',15,'bold'))
    extractBtn.pack()
def upload():
    try:
        path=filedialog.askopenfilename()
        image=Image.open(path)
        img=ImageTk.PhotoImage(image)
        uploaded_img.configure(image=img)
        uploaded_img.image=img
        show_extract_button(path)
    except:
        pass  
uploadbtn = Button(root,text="Upload an image",command=upload,bg="#2f2f77",fg="gray",height=2,width=20,font=('Times',15,'bold')).pack()
newline.configure(text='\n')
newline.pack()
uploaded_img.pack()
root.mainloop()

Yt 
was 
the best of 
times, it 
was 
the 
worst 
of times, it 
was 
the 
age 
of wisdom, it 
was 
the 
Yt 
was 
the best of 
times, it 
was 
the 
worst 
of times, it 
was 
the 
age 
of wisdom, it 
was 
the 


In [8]:
from PIL import Image
from pytesseract import pytesseract
path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
image_path = r"C:\Users\jayen\OneDrive\Desktop\1.PNG"
img = Image.open(image_path)
pytesseract.tesseract_cmd = path_to_tesseract
text = pytesseract.image_to_string(img)
print(text[:-1])
